In [1]:
def asistencia(materia, sesion):
    from pydub import AudioSegment
    from pydub.playback import play

    from matplotlib.pyplot import figure, ion, pause, axis
    from matplotlib.patches import Rectangle
    from time import sleep

    import cv2
    import signal
    from pyzbar.pyzbar import decode, ZBarSymbol
    
    from yaml import safe_load, YAMLError, safe_dump
    
    def signal_handler(signal, frame):
        # KeyboardInterrupt detected, exiting
        global is_interrupted
        is_interrupted = True
        
    vc = cv2.VideoCapture(0)

    fig = figure()
    ax = fig.gca()
    ion()
    axis('off')
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    fig.canvas.draw()
    song = AudioSegment.from_wav("Frog.wav")

    if vc.isOpened(): # try to get the first frame
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ax.clear()
        #datos = decode(frame, symbols=[ZBarSymbol.CODE39])
        webcam_preview = ax.imshow(cv2.flip(frame, 1), cmap='gray')
        fig.canvas.draw()
    else:
        is_capturing = False

    signal.signal(signal.SIGINT, signal_handler)
    is_interrupted = False

    codigos = []
    cuadros = []

    while is_capturing:
        is_capturing, frame = vc.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        datos = decode(frame, symbols=[ZBarSymbol.CODE39])
        [p.remove() for p in reversed(ax.patches)]
        webcam_preview.set_data(cv2.flip(frame, 1))

        if len(datos) > 0:
            play(song)
            datos_r = datos[0][2]
            cod = datos[0][0]
            rect = Rectangle((1280-datos_r[0], datos_r[1]), -datos_r[2], datos_r[3],
                             linewidth=1, edgecolor=(0,1,0), facecolor=(0,1,0,0.3))
            ax.add_patch(rect)
            codigos.append(cod)
            cuadros.append(frame)
            fig.canvas.draw()
            sleep(2)

        fig.canvas.draw()

        try:
            pause(0.01)
        except Exception:
            pass
        if is_interrupted:
            vc.release()
            break
    
    with open(materia+str(sesion), "w") as outfile:
        safe_dump({"cuentas": codigos}, outfile,
                  encoding='utf-8', allow_unicode=True, default_flow_style=False)
    return codigos

In [ ]:
%matplotlib widget
asistencia("Interfaces y perifericos para robots", 2)

FigureCanvasNbAgg()